## Inputs and output


In [1]:
fname_dblp = "../inputs/DBLP1.csv"
fname_scholar = "../inputs/Scholar.csv"
fname_result = "../outputs/DBLP_Scholar_perfectMapping_GuofengZhao.csv"

## Import required packages
Difflib - Ratcliff/Obershelp algorithm
distance - Measure Levenshtein/Hamming/Jaccard/Sorensen distance

In [2]:
import pandas as pd
import numpy as np
import re
import difflib
import distance
from datetime import datetime

title_similarity_threshold = 0.90
authors_similarity_threshold = 0.50
venue_similarity_threshold = 0.90
year_similarity_threshold = 0.99

## load DBLP csv files into data frame

In [3]:
dblp = pd.read_csv(fname_dblp, quotechar='"', encoding="ansi", error_bad_lines=False)


## Cleaning DBLP

In [4]:
print ("Total DBLP rows: {}".format(dblp.shape))
print(dblp.count())
#dblp.head(5)
dblp['title'] = dblp['title'].map(lambda x: x if type(x)!=str else x.lower().strip())
dblp['authors'] = dblp['authors'].map(lambda x: x if type(x)!=str else x.lower().strip())
dblp['venue'] = dblp['venue'].map(lambda x: x if type(x)!=str else x.lower().strip())

dblp.head(5)


Total DBLP rows: (2615, 6)
idDBLP     2615
title      2615
authors    2397
venue      2404
year       2615
Row_ID     2615
dtype: int64


,idDBLP,title,authors,venue,year,Row_ID
0,journals/sigmod/EisenbergM02,sql/xml is making good progress,"a eisenberg, j melton",sigmod record,2002,2
1,conf/vldb/AmmannJR95,using formal methods to reason about semantics...,"p ammann, s jajodia, i ray",vldb,1995,3
2,journals/sigmod/Liu02,editor's notes,l liu,sigmod record,2002,4
3,journals/sigmod/Hammer02,report on the acm fourth international worksho...,NaN,NaN,2002,5
4,conf/vldb/FerrandinaMZFM95,schema and database evolution in the o2 object...,"f ferrandina, t meyer, r zicari, g ferran, j m...",vldb,1995,6


## Deduplicate for DBLP

In [5]:
# define a few helper functions
def title_similarity_difflib (row1, row2):
    return difflib.SequenceMatcher(None, row1[1], row2[1]).quick_ratio()

def title_similarity_levenshtein (row1, row2):
    return 1- distance.nlevenshtein(row1[1], row2[1], method=1)

def title_similarity_sorensen (row1, row2):
    return 1- distance.sorensen(row1[1], row2[1])

def title_similarity_jaccard (row1, row2):
    return 1- distance.jaccard(row1[1], row2[1])

def title_similarity(row1, row2, method = "difflib"):
    if method.lower() == "levenshtein":
        return title_similarity_levenshtein (row1, row2)
    if method.lower() == "sorensen":
        return title_similarity_sorensen (row1, row2)
    if method.lower() == "jaccard":
        return title_similarity_jaccard (row1, row2)
    return title_similarity_difflib (row1, row2)
    
def authors_similarity (row1, row2):
    if pd.isnull(row1[2]):
        return 1
    if pd.isnull(row2[2]):
        return 1
    return difflib.SequenceMatcher(None, re.split(r'[;,\s]\s*', row1[2]), re.split(r'[;,\s]\s*', row2[2])).quick_ratio()
def venue_similarity (row1, row2):
    if pd.isnull(row1[3]):
        return 1
    if pd.isnull(row2[3]):
        return 1
    return difflib.SequenceMatcher(None, row1[3], row2[3]).quick_ratio()

def year_similarity (row1, row2):
    if pd.isnull(row1[4]):
        return 1
    if pd.isnull(row2[4]):
        return 1
    return 1 if row1[4] == row2[4] else 0

def row_similar (row1, row2):
    if title_similarity(row1, row2) < title_similarity_threshold:
        return False
    if authors_similarity(row1, row2) < authors_similarity_threshold:
        return False
    if venue_similarity(row1, row2) < venue_similarity_threshold:
        return False
    if year_similarity(row1, row2) < year_similarity_threshold:
        return False
    return True

In [8]:
def deduplicate (data):
    data['duplicate'] = -1
    data_columns = data.axes[1].values
    data_values = data.values
    nrows, ncolumns = data_values.shape
    print("Start calculating similarity - " + str(datetime.now()))
    count = 0
    for i in range(nrows):
        for j in range(i+1, nrows):
            count = count+1
            if count % 1000000 == 0:
                print("{} comparison calculated.".format(count))
            if row_similar(data_values[i, :], data_values[j, :]):
                data_values[i, 6] = data_values[j, 5]
                if pd.isnull(data_values[j, 2]):
                    data_values[j, 2] = data_values[i, 2]
                if pd.isnull(data_values[j, 3]):
                    data_values[j, 3] = data_values[i, 3]
                if pd.isnull(data_values[j, 2]):
                    data_values[j, 4] = data_values[i, 4]
                break
    print("{} comparison calculated.".format(count))
    print("Finished calculating similarity - " + str(datetime.now()))
    tmp = pd.DataFrame(data = data_values, columns = data_columns)
    dupe_rows = tmp[tmp["duplicate"] >= 0]
    print("duplicates: {}".format(dupe_rows.shape))
    unique_rows = tmp[tmp["duplicate"] < 0]
    print("unique rows kept: {}".format(unique_rows.shape))
    return dupe_rows, unique_rows

In [9]:
dblp_dupe, dblp_unique = deduplicate(dblp)


Start calculating similarity - 2018-03-09 01:51:11.065066
1000000 comparison calculated.
2000000 comparison calculated.
3000000 comparison calculated.
3311631 comparison calculated.
Finished calculating similarity - 2018-03-09 01:53:50.796687
duplicates: (65, 7)
unique rows kept: (2550, 7)


In [ ]:
dblp_dupe.head(10)

In [ ]:
dblp[dblp['Row_ID'] == 1357].loc[:, 'authors']


In [ ]:
dblp[dblp['Row_ID'] == 1489].loc[:, 'authors']

## load Scholar csv into data frame

In [10]:
scholar = pd.read_csv(fname_scholar, quotechar='"', encoding="ansi", engine="python")
#scholar.rename(columns={'ROW_ID':'Row_ID'}, inplace = True)

## cleaning Scholar

In [11]:
print ("Total Scholar rows: {}".format(scholar.shape))
print(scholar.count())

scholar['title'] = scholar['title'].map(lambda x: x if type(x)!=str else x.lower().strip())
scholar['authors'] = scholar['authors'].map(lambda x: x if type(x)!=str else x.lower().strip())
scholar['venue'] = scholar['venue'].map(lambda x: x if type(x)!=str else x.lower().strip())
scholar.head(5)

Total Scholar rows: (64260, 6)
idScholar    64260
title        64260
authors      64259
venue        49263
year         29471
ROW_ID       64260
dtype: int64


,idScholar,title,authors,venue,year,ROW_ID
0,aKcZKwvwbQwJ,11578 sorrento valley road,qd inc,"san diego,",NaN,1
1,ixKfiTHoaDoJ,initiation of crazes in polystyrene,"as argon, jg hannoosh","phil. mag,",NaN,2
2,3BxllB4wwcIJ,immunogold labelling is a quantitative method ...,"gh hansen, ll wetterberg, h sjí€œ_strí€œ_m, o ...","the histochemical journal,",1992.0,3
3,d2WWxwKMex4J,the burden of infectious disease among inmates...,"tm hammett, p harmon, w rhodes",see,NaN,4
4,cZCX-AQpjccJ,the role of faculty advising in science and en...,jr cogdell,"new directions for teaching and learning,",1995.0,5


In [ ]:
scholar_dupe, scholar_unique = deduplicate(scholar)

Start calculating similarity - 2018-03-09 01:54:27.712164
1000000 comparison calculated.
2000000 comparison calculated.
3000000 comparison calculated.
4000000 comparison calculated.
5000000 comparison calculated.
6000000 comparison calculated.
7000000 comparison calculated.
8000000 comparison calculated.
9000000 comparison calculated.
10000000 comparison calculated.
11000000 comparison calculated.
12000000 comparison calculated.
13000000 comparison calculated.
14000000 comparison calculated.
15000000 comparison calculated.
16000000 comparison calculated.
17000000 comparison calculated.
18000000 comparison calculated.
19000000 comparison calculated.
20000000 comparison calculated.
21000000 comparison calculated.
22000000 comparison calculated.
23000000 comparison calculated.
24000000 comparison calculated.
25000000 comparison calculated.
26000000 comparison calculated.
27000000 comparison calculated.
28000000 comparison calculated.
29000000 comparison calculated.
30000000 comparison cal

## link records from DBLP to Scholar

In [ ]:
scholar_dupe.head(10)

In [ ]:
scholar[scholar['Row_ID'] == 1489].loc[:, 'authors']

In [ ]:
# generate the N*M pair matrix
scholar['tmp'] = 1
#max_row_id = scholar.loc[:, ["Row_ID"]].max().at["Row_ID"]
dblp_scholar = dblp_clean.merge(scholar, on='tmp', suffixes=["","_R"])
print("Total pairs after merge: {}".format(dblp_scholar.shape))
dblp_scholar.drop(labels=['tmp'], axis=1, inplace=True)
#dblp_scholar = dblp_scholar[(dblp_scholar["Row_ID"] < dblp_scholar["Row_ID_R"]) | (dblp_scholar["Row_ID_R"] == max_row_id)]
#print("Total pairs after half-fold deduplicate: {}".format(scholar_pair.shape))
print("Start calculating similarity - " + str(datetime.now()))
dblp_scholar['row_similar'] = dblp_scholar.apply(row_similar, axis = 1)
print("Finished calculating similarity - " + str(datetime.now()))
print("Total pairs after similarity evaluation: {}".format(dblp_scholar.shape))
dblp_scholar.head(5)